In [ ]:
from pathlib import Path
import os
from collections import deque

In [ ]:
fp = os.path.join(Path().absolute(), "inputs", "input25.txt")
# fp = os.path.join(Path().absolute(), "inputs", "input25_test.txt")

with open(fp, "r") as f:
    data = f.read().split("\n")[:-1]

In [ ]:
data

In [ ]:
all_nodes = []

for line in data:
    from_node, to_nodes = line.split(": ")
    to_nodes = to_nodes.split()
    all_nodes.append(from_node)
    all_nodes += to_nodes

all_nodes = list(set(all_nodes))

In [ ]:
all_nodes

In [ ]:
num_nodes = len(all_nodes)
num_nodes

In [ ]:
adj_matrix = [[0 for _ in range(num_nodes)] for _ in range(num_nodes)]

for line in data:
    from_node, to_nodes = line.split(": ")
    to_nodes = to_nodes.split()
    
    from_node_index = all_nodes.index(from_node)

    for to_node in to_nodes:
        to_node_index = all_nodes.index(to_node)
        adj_matrix[from_node_index][to_node_index] = 1
        adj_matrix[to_node_index][from_node_index] = 1

In [ ]:
sum(sum(row) for row in adj_matrix)

In [ ]:
[sum(row) for row in adj_matrix]

In [ ]:
adj_matrix_sparse = []
for i in range(num_nodes):
    adj_matrix_sparse.append([j for j, val in enumerate(adj_matrix[i]) if val == 1])

In [ ]:
adj_matrix_sparse

In [ ]:
def bfs(adj_matrix_sparse, start_idx, residual_capacity):

    # Stores path
    parents = [None] * num_nodes

    visited = [False] * num_nodes
    queue = deque()

    queue.append(start_idx)
    visited[start_idx] = True

    while queue:
        u = queue.popleft()
        for v in adj_matrix_sparse[u]:
            if not visited[v] and residual_capacity[u][v] > 0:
                queue.append(v)
                visited[v] = True
                parents[v] = u

    return visited, parents

In [ ]:
source_idx = 0

for sink_idx in range(num_nodes):
    if sink_idx == source_idx:
        continue

    print(f"{sink_idx = }")

    # Run Ford-Fulkerson
    capacity_matrix = adj_matrix

    # Initialise flows as zero
    flow_matrix = [[0 for _ in range(num_nodes)] for _ in range(num_nodes)]

    while True:
        # Find residual network
        residual_capacity = [[None for _ in range(num_nodes)] for _ in range(num_nodes)]
        for i in range(num_nodes):
            for j in range(num_nodes):
                residual_capacity[i][j] = capacity_matrix[i][j] - flow_matrix[i][j]

        # Find augmenting path
        # print("Finding augmenting path")
        visited, parents = bfs(adj_matrix_sparse, source_idx, residual_capacity)

        if not visited[sink_idx]:
            # print("No more augmenting paths")
            # No more augmenting paths - done
            max_flow = sum(flow_matrix[source_idx])
            print(f"{max_flow = }")

            S = [i for i, val in enumerate(visited) if val == True]
            edges = []
            for i in S:
                for j in adj_matrix_sparse[i]:
                    if j not in S:
                        edges.append((i, j))

            min_cut_weight = len(edges)
            print(f"{min_cut_weight = }")
            assert max_flow == min_cut_weight
            
            if min_cut_weight == 3:
                group_one_size = len(S)
                group_two_size = num_nodes - len(S)
                result = group_one_size * group_two_size
                print("Finished, result = ", result)
            elif min_cut_weight < 2:
                raise ValueError

            break

        else:
            # iterate through edges in augmenting path
            current = sink_idx
            while True:
                parent = parents[current]
                res_cap = residual_capacity[parent][current]

                flow_matrix[parent][current] += res_cap
                flow_matrix[current][parent] -= res_cap

                current = parent
                if current == source_idx:
                    break